In [ ]:
# -*- coding: utf-8 -*-
"""
Binary IVDD training (ivdd vs normal) from DeepLabCut CSV (3-level header)
要件:
- model/ フォルダに学習モデルを毎回ユニーク名で保存（best と final）
- fig/ フォルダに学習曲線（loss/accuracy）を毎回ユニーク名で保存
- 前処理: tail_set を原点に平行移動 → 各次元を min-max 正規化（ファイル単位）
- EarlyStopping は使わない
- ネットワーク構造: TimeDistributed(Dense->ReLU) → LSTM → LSTM → Dense(1 logits)（従来通り）
"""

# ===== 安定運用: GPU無効/ログ控えめ/スレッド抑制（必要なら） =====
import os
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "-1")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

import glob, re, json, datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

print("TF:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

# ========= パラメータ =========
# 学習用CSVを置いたフォルダ（例: train ディレクトリ）
DATA_DIR   = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\ivdd\train"
CSV_GLOB   = os.path.join(DATA_DIR, "*.csv")

# 保存先（要件）
MODEL_DIR = os.path.join(DATA_DIR, "model")
FIG_DIR   = os.path.join(DATA_DIR, "fig")
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)

# ランごとユニークID
RUN_ID = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 指定の5点（順序がそのまま (x,y) の並び順になります）
KEYPOINTS = [
    "left back paw",
    "right back paw",
    "left front paw",
    "right front paw",
    "tail set",
]

# DLCのlikelihood閾値で低信頼を欠損扱いに
USE_LIKELIHOOD      = False
MIN_KEEP_LIKELIHOOD = 0.6

# ウィンドウ設定（必要に応じて 60/30 などに変更）
SEQ_LEN  = 60
STRIDE   = 30

# 次元・ネットワーク
DIMS      = 10           # 5点×(x,y)
N_HIDDEN  = 30
N_CLASSES = 1            # バイナリ → ロジット1本

# 学習ハイパパラメータ
BATCH_SIZE = 32
EPOCHS     = 50
LR         = 1e-3        # floatにしておくと ReduceLROnPlateau が適用可能
L2_LAMBDA  = 1e-4

# 分割設定
VAL_SPLIT_BY_FILE = True   # ファイル単位で分割（時系列リーク防止）

# ラベル表記（ivdd を positive=1 として扱います）
CLASS_NAMES  = ["normal", "ivdd"]  # index 0=normal, 1=ivdd
CLASS_TO_IDX = {"normal": 0, "ivdd": 1}

# ========= ラベル推定（ファイル名から） =========
def infer_label_from_filename(path: str) -> int:
    """
    ファイル名からラベルを推定。
    1) stem を非英数字で分割したトークンに 'ivdd' / 'normal' が厳密一致すれば採用
    2) 未決なら先頭トークンが 'ivdd' / 'normal' なら採用
    3) まだ未決なら親ディレクトリ名トークンを参照（片方のみ含む場合）
    """
    name = os.path.basename(path).lower()
    stem = os.path.splitext(name)[0]
    tokens = [t for t in re.split(r'[^a-z0-9]+', stem) if t]
    token_set = set(tokens)

    has_ivdd   = ('ivdd' in token_set)
    has_normal = ('normal' in token_set)

    if has_ivdd and not has_normal:
        return CLASS_TO_IDX["ivdd"]
    if has_normal and not has_ivdd:
        return CLASS_TO_IDX["normal"]

    if tokens:
        if tokens[0] in CLASS_TO_IDX:
            return CLASS_TO_IDX[tokens[0]]

    parent_tokens = [t for t in re.split(r'[^a-z0-9]+', os.path.dirname(path).lower()) if t]
    pset = set(parent_tokens)
    if ('ivdd' in pset) and ('normal' not in pset):
        return CLASS_TO_IDX["ivdd"]
    if ('normal' in pset) and ('ivdd' not in pset):
        return CLASS_TO_IDX["normal"]

    raise ValueError(
        f"ラベルを特定できません: {name} "
        f"(推奨: ファイル名の先頭を 'ivdd_' または 'normal_' にしてください)"
    )

# ========= 前処理（DLC 3 レベルヘッダ） =========
def _norm_name(s: str) -> str:
    return "".join(ch for ch in s.lower() if ch not in " _-")

def _resolve_keypoints(all_bodyparts, requested):
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_name(bp)
        if k not in norm2orig:
            norm2orig[k] = bp
    resolved, missing = [], []
    for req in requested:
        k = _norm_name(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)
    if missing:
        raise ValueError(f"指定キーポイントがCSVで見つかりません: {missing}\n利用可能: {all_bodyparts}")
    return resolved

def read_dlc_5kp_xy(csv_path: str,
                    keypoints,
                    use_likelihood=True,
                    min_keep_likelihood=0.6):
    """
    (1) DLC CSV(3段ヘッダ)読込 → (2) 指定5点の (x,y) 抜き出し → (3) 低likelihoodをNaN
       → (4) 線形補間 + bfill/ffill → 0埋め → (T,10) を返す
    """
    df = pd.read_csv(csv_path, header=[0,1,2], index_col=0)
    bodyparts = list({bp for (_, bp, _) in df.columns})
    use_kps = _resolve_keypoints(bodyparts, keypoints)

    cols = {}
    for bp in use_kps:
        cols[f"{bp}_x"] = df.xs((bp, "x"), level=[1,2], axis=1)
        cols[f"{bp}_y"] = df.xs((bp, "y"), level=[1,2], axis=1)
    X_df = pd.concat(cols.values(), axis=1)
    X_df.columns = list(cols.keys())

    if use_likelihood:
        for bp in use_kps:
            try:
                lcol = df.xs((bp, "likelihood"), level=[1,2], axis=1).values.flatten()
                low = lcol < min_keep_likelihood
                for c in [f"{bp}_x", f"{bp}_y"]:
                    v = X_df[c].values
                    v[low] = np.nan
                    X_df[c] = v
            except KeyError:
                pass

    X_df = X_df.interpolate(method="linear", limit_direction="both", axis=0)
    # FutureWarning 回避: bfill()/ffill() を使う
    X_df = X_df.bfill().ffill().fillna(0.0)

    X = X_df.values.astype(np.float32)  # (T, 10)
    return X, use_kps

# === tail_set中心 + min-max 正規化（ファイル単位・各次元独立）===
def normalize_tailset_minmax(
    X: np.ndarray,         # (T,10)
    used_kps: list[str],   # read_dlc_5kp_xy で実際に使われた実名
    ref_name: str = "tail set",
    eps: float = 1e-6
) -> np.ndarray:
    """
    1) tail_setを原点に平行移動（各フレーム）
    2) 平行移動後の各次元(全フレーム)で min-max 正規化（0〜1）
       x' = (x - min) / (max - min + eps)
    """
    low = [s.lower() for s in used_kps]
    if ref_name.lower() not in low:
        raise ValueError(f"'{ref_name}' が used_kps に見つかりません: {used_kps}")
    ref_idx = low.index(ref_name.lower())

    # 1) 原点平行移動
    cx = X[:, 2*ref_idx]
    cy = X[:, 2*ref_idx + 1]
    Xc = X.copy()
    for i in range(len(used_kps)):
        Xc[:, 2*i]   -= cx
        Xc[:, 2*i+1] -= cy

    # 2) min-max 正規化（ファイル全体、各次元独立）
    x_min = Xc.min(axis=0, keepdims=True)
    x_max = Xc.max(axis=0, keepdims=True)
    Xn = (Xc - x_min) / (x_max - x_min + eps)
    return Xn

def make_windows(X: np.ndarray, seq_len: int, stride: int) -> np.ndarray:
    n = X.shape[0]
    if n < seq_len:
        return np.empty((0, seq_len, X.shape[1]), dtype=X.dtype)
    starts = range(0, n - seq_len + 1, stride)
    return np.stack([X[s:s+seq_len] for s in starts], axis=0)

def build_dataset(csv_paths, seq_len=SEQ_LEN, stride=STRIDE):
    """
    返り値:
      X: (N, T, D)
      y: (N,) 0/1
      file_ids: (N,) 元ファイル名
    """
    X_list, y_list, file_ids = [], [], []
    used_kps_any = None

    for p in csv_paths:
        y_lab = infer_label_from_filename(p)  # 0=normal,1=ivdd

        X_raw, used_kps = read_dlc_5kp_xy(
            p, keypoints=KEYPOINTS,
            use_likelihood=USE_LIKELIHOOD,
            min_keep_likelihood=MIN_KEEP_LIKELIHOOD
        )
        if used_kps_any is None:
            used_kps_any = used_kps

        if X_raw.shape[1] != DIMS:
            raise ValueError(f"{os.path.basename(p)}: 次元 {X_raw.shape[1]} != 期待 {DIMS}")

        # ★ tail_set中心 + min-max 正規化
        X_norm = normalize_tailset_minmax(X_raw, used_kps, ref_name="tail set")

        X_win = make_windows(X_norm, seq_len, stride)  # (M, T, D)
        if X_win.shape[0] == 0:
            print(f"[WARN] {os.path.basename(p)}: フレーム不足（{seq_len}未満）でスキップ")
            continue

        X_list.append(X_win)
        y_list.append(np.full((X_win.shape[0],), y_lab, dtype=np.int64))
        file_ids += [os.path.basename(p)] * X_win.shape[0]

    if not X_list:
        raise RuntimeError("データが作れませんでした。CSV名に 'ivdd' / 'normal' が含まれているか確認してください。")

    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    file_ids = np.array(file_ids)
    print(f"[INFO] 使用キーポイント実名: {used_kps_any}")
    return X, y, file_ids

# ========= モデル定義（Functional; 構造は従来通り） =========
def build_lstm_model(seq_len: int, dims: int, n_hidden: int, l2_lambda: float = 1e-4) -> keras.Model:
    reg = keras.regularizers.l2(l2_lambda)
    inp = keras.Input(shape=(seq_len, dims), name="input")
    td  = keras.layers.TimeDistributed(keras.layers.Dense(n_hidden, activation="relu", kernel_regularizer=reg),
                                       name="td_dense")(inp)
    x   = keras.layers.LSTM(n_hidden, return_sequences=True, kernel_regularizer=reg, name="lstm1")(td)
    x   = keras.layers.LSTM(n_hidden, kernel_regularizer=reg, name="lstm2")(x)
    out = keras.layers.Dense(1, kernel_regularizer=reg, name="logits")(x)  # from_logits=True
    model = keras.Model(inp, out, name="ivdd_lstm")
    return model

# ========= データ読み込み =========
csv_files = sorted(glob.glob(CSV_GLOB))
if not csv_files:
    raise FileNotFoundError(f"CSV が見つかりません: {CSV_GLOB}")

X, y, file_ids = build_dataset(csv_files, SEQ_LEN, STRIDE)
print("X:", X.shape, "y:", y.shape, "files:", len(np.unique(file_ids)))

# ========= 分割（ファイル単位推奨） =========
if VAL_SPLIT_BY_FILE:
    uniq = np.unique(file_ids)
    tr_files, va_files = train_test_split(uniq, test_size=0.2, random_state=42, shuffle=True)
    tr_mask = np.isin(file_ids, tr_files)
    va_mask = np.isin(file_ids, va_files)
    X_train, y_train = X[tr_mask], y[tr_mask]
    X_val,   y_val   = X[va_mask], y[va_mask]
else:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("train:", X_train.shape, "val:", X_val.shape)

# ========= クラス不均衡対策 (0/1 の重み) =========
# 片方しか存在しない場合は class_weight を None に
unique_classes = np.unique(y_train)
if set(unique_classes) == {0, 1}:
    cls_w = compute_class_weight(class_weight="balanced", classes=np.array([0,1]), y=y_train)
    class_weight = {0: float(cls_w[0]), 1: float(cls_w[1])}
else:
    class_weight = None
print("class_weight:", class_weight)

# ========= モデル準備・学習（EarlyStopping なし） =========
model = build_lstm_model(SEQ_LEN, DIMS, N_HIDDEN, L2_LAMBDA)
model.summary()

opt = keras.optimizers.Adam(learning_rate=LR)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [keras.metrics.BinaryAccuracy(name="accuracy")]
model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)

# チェックポイント（best のみ）※要件: model/ にユニーク名で保存
best_model_path  = os.path.join(MODEL_DIR, f"ivdd_lstm_{RUN_ID}_best.keras")
final_model_path = os.path.join(MODEL_DIR, f"ivdd_lstm_{RUN_ID}_final.keras")

callbacks = [
    keras.callbacks.ModelCheckpoint(
        best_model_path, monitor="val_accuracy",
        save_best_only=True, save_weights_only=False, mode="max", verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min",
        factor=0.5, patience=5, min_lr=1e-5, verbose=1
    ),
    # ★ EarlyStopping は使わない（要件）
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weight,
    verbose=1,
    callbacks=callbacks
)

# ========= 学習曲線を fig/ にユニーク名で保存 =========
loss_png = os.path.join(FIG_DIR, f"loss_{RUN_ID}.png")
acc_png  = os.path.join(FIG_DIR, f"accuracy_{RUN_ID}.png")

plt.figure(figsize=(8,4))
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss")
plt.legend(); plt.tight_layout()
plt.savefig(loss_png, dpi=150); plt.close()

plt.figure(figsize=(8,4))
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy")
plt.legend(); plt.tight_layout()
plt.savefig(acc_png, dpi=150); plt.close()

print("[INFO] saved figs:", loss_png, acc_png)

# ========= 検証セットでの簡易評価（ウィンドウ単位） =========
logits_val = model.predict(X_val, batch_size=64)
y_pred = (tf.math.sigmoid(logits_val).numpy().ravel() >= 0.5).astype(int)

print("\n[Window-level] classification_report:")
print(classification_report(y_val, y_pred, target_names=CLASS_NAMES, digits=4))
cm = confusion_matrix(y_val, y_pred)
print("[Window-level] confusion matrix:\n", cm)

# ========= 最終モデルを保存（model/ にユニーク名） =========
model.save(final_model_path)
print("[INFO] saved final model to:", final_model_path)

# # （任意）学習に使った主要パラメータも一緒に保存しておくと後で便利
# params_json = os.path.join(MODEL_DIR, f"train_params_{RUN_ID}.json")
# with open(params_json, "w", encoding="utf-8") as f:
#     json.dump({
#         "RUN_ID": RUN_ID,
#         "KEYPOINTS": KEYPOINTS,
#         "USE_LIKELIHOOD": USE_LIKELIHOOD,
#         "MIN_KEEP_LIKELIHOOD": MIN_KEEP_LIKELIHOOD,
#         "SEQ_LEN": SEQ_LEN,
#         "STRIDE": STRIDE,
#         "DIMS": DIMS,
#         "N_HIDDEN": N_HIDDEN,
#         "L2_LAMBDA": L2_LAMBDA,
#         "CLASS_NAMES": CLASS_NAMES,
#         "VAL_SPLIT_BY_FILE": VAL_SPLIT_BY_FILE,
#         "BATCH_SIZE": BATCH_SIZE,
#         "EPOCHS": EPOCHS,
#         "LR": LR,
#         "DATA_DIR": DATA_DIR,
#         "CSV_GLOB": CSV_GLOB
#     }, f, ensure_ascii=False, indent=2)

# print("[INFO] saved params to:", params_json)
print("[DONE] training complete.")


TF: 2.19.0
[WARN] ivdd1_case1DLC_resnet50_IvddOct30shuffle1_100000.csv: フレーム不足（60未満）でスキップ
[INFO] 使用キーポイント実名: ['left back paw', 'right back paw', 'left front paw', 'right front paw', 'tail set']
X: (1236, 60, 10) y: (1236,) files: 170
train: (979, 60, 10) val: (257, 60, 10)
class_weight: {0: 1.1049661399548534, 1: 0.9132462686567164}


Model: "ivdd_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 60, 10)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ td_dense (TimeDistributed)      │ (None, 60, 30)         │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm1 (LSTM)                    │ (None, 60, 30)         │         7,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm2 (LSTM)                    │ (None, 30)             │         7,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,001 (58.60 KB)

 Trainable params: 15,001 (58.60 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4271 - loss: 0.7014
Epoch 1: val_accuracy improved from -inf to 0.38132, saving model to C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\ivdd\train\model\ivdd_lstm_20251124-092956_best.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.4279 - loss: 0.7014 - val_accuracy: 0.3813 - val_loss: 0.7242 - learning_rate: 0.0010
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4271 - loss: 0.7010
Epoch 2: val_accuracy did not improve from 0.38132
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4279 - loss: 0.7010 - val_accuracy: 0.3813 - val_loss: 0.7038 - learning_rate: 0.0010
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4306 - loss: 0.6890
Epoch 3: val_accuracy improved from 0.38132 to 0.40078, saving model to C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-mas